<a href="https://colab.research.google.com/github/shintaroZ/iot-tool/blob/main/fast_stable_diffusion_ComfyUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ComfyUI notebook by https://github.com/TheLastBen/fast-stable-diffusion**

In [1]:
#@markdown # Connect Google Drive
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
Shared_Drive = "" #@param {type:"string"}
#@markdown - Leave empty if you're not using a shared drive

print("[0;33mConnecting...")
drive.mount('/content/gdrive')

if Shared_Drive!="" and os.path.exists("/content/gdrive/Shareddrives"):
  mainpth="Shareddrives/"+Shared_Drive
else:
  mainpth="MyDrive"

clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [2]:
#@markdown # Install/Update ComfyUI repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time
import base64
import requests
from urllib.request import urlopen, Request
from urllib.parse import urlparse, parse_qs, unquote
from tqdm import tqdm
import six


if not os.path.exists("/content/gdrive"):
  print('[1;31mGdrive not connected, using temporary colab storage ...')
  time.sleep(4)
  mainpth="MyDrive"
  !mkdir -p /content/gdrive/$mainpth
  Shared_Drive=""

if Shared_Drive!="" and not os.path.exists("/content/gdrive/Shareddrives"):
  print('[1;31mShared drive not detected, using default MyDrive')
  mainpth="MyDrive"

with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  !git clone -q --depth 1 --branch main https://github.com/TheLastBen/diffusers
  %cd /content/gdrive/$mainpth/
  !git clone --depth 1 https://github.com/comfyanonymous/ComfyUI
  !mkdir -p /content/gdrive/$mainpth/sd/cache/
  os.environ['TRANSFORMERS_CACHE']=f"/content/gdrive/{mainpth}/sd/cache"
  os.environ['TORCH_HOME'] = f"/content/gdrive/{mainpth}/sd/cache"

with capture.capture_output() as cap:
  %cd /content/gdrive/$mainpth/ComfyUI
  !git reset --hard
  time.sleep(1)
  !git pull
clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [ ]:
#@markdown # Requirements

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  !pip uninstall wandb -y
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt

  if not os.path.exists('gdrive/'+mainpth+'/sd/libtcmalloc/libtcmalloc_minimal.so.4'):
    %env CXXFLAGS=-std=c++14
    !wget -q https://github.com/gperftools/gperftools/releases/download/gperftools-2.5/gperftools-2.5.tar.gz && tar zxf gperftools-2.5.tar.gz && mv gperftools-2.5 gperftools
    !wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/Patch
    %cd /content/gperftools
    !patch -p1 < /content/Patch
    !./configure --enable-minimal --enable-libunwind --enable-frame-pointers --enable-dynamic-sized-delete-support --enable-sized-delete --enable-emergency-malloc; make -j4
    !mkdir -p /content/gdrive/$mainpth/sd/libtcmalloc && cp .libs/libtcmalloc*.so* /content/gdrive/$mainpth/sd/libtcmalloc
    %env LD_PRELOAD=/content/gdrive/$mainpth/sd/libtcmalloc/libtcmalloc_minimal.so.4
    %cd /content
    !rm *.tar.gz Patch && rm -r /content/gperftools
  else:
    %env LD_PRELOAD=/content/gdrive/$mainpth/sd/libtcmalloc/libtcmalloc_minimal.so.4

  !pip install wandb==0.15.12 pydantic==2.10.5 numpy==1.26 scipy==1.15.3 -qq
  !pip install diffusers accelerate transformers av comfyui-frontend-package comfyui-workflow-templates alembic -U -qq
  !rm -r /usr/local/lib/python3.12/dist-packages/tensorflow*
  os.environ['PYTHONWARNINGS'] = 'ignore'
  !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.12/warnings.py
  !sed -i 's@raise AttributeError(f"module {module!r} has no attribute {name!r}")@@g' /usr/local/lib/python3.12/dist-packages/jax/_src/deprecations.py
  !sed -i 's@globalns, localns, set()@globalns, localns, recursive_guard=set()@g' /usr/local/lib/python3.12/dist-packages/pydantic/typing.py

clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Installing requirements...


In [ ]:
#@markdown # Start ComfyUI
from IPython.utils import capture
import time
import sys
import fileinput
from pyngrok import ngrok, conf
import re
from subprocess import call


Ngrok_Token = "" #@param {type:"string"}

#@markdown - Ngrok token must be entered to be able to run ComfyUI

#ngrok.kill()
#time.sleep(2)
with capture.capture_output() as cap:
  localurl=ngrok.connect(666, pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_Token) , bind_tls=True).public_url
  !rm -r /content/gdrive/MyDrive/ComfyUI/custom_nodes/.ipynb_checkpoints
call("sed -i 's@^            if verbose:@@' /content/gdrive/MyDrive/ComfyUI/server.py", shell=True)
call("sed -i 's@^                logging.info(\"To see the GUI go to: {}://{}:{}\".format(scheme, address_print, port))@        logging.info(\"[32m\u2714 Connected\");logging.info(\"[1;34m"+localurl+"[0m\")@' /content/gdrive/MyDrive/ComfyUI/server.py", shell=True)
!sed -i 's@https:.*@{localurl}[0m\")@' /content/gdrive/MyDrive/ComfyUI/server.py
!python /content/gdrive/MyDrive/ComfyUI/main.py --listen --port 666 #--preview-method auto